In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## Business problem

# Data Description

A dataset has been extracted containing container calls at terminals that took place during the past 3
years for the Port of Hamburg and Port of Los Angeles. The dataset contains the following fields

In [3]:
raw_data = pd.read_csv("data/mt_terminal_calls.csv")

A dataset has been extracted containing container calls at terminals that took place during the past 3
years for the Port of Hamburg and Port of Los Angeles. The dataset contains the following fields
* **last_port**: port where the last terminal call by vessel is recorded
* **last_terminal**: the immediately previous terminal call of the vessel
* **last_terminal_doc_timestamp**: timestamp of previous terminal call
* **current_port**: port where the current terminal call by vessel is recorded
* **current_terminal**: current terminal call of the vessel
* **shipname**: name of the vessel
* **dock_timestamp**: timestamp of current terminal call
* **GRT**: vessel capacity (gross tonnage unit)
* **TEU**: vessel capacity (twenty-foot equivalent Unit)
* **length**: vessel length
* **width**: vessel width

In [5]:
raw_data.head(5)

,last_port,last_terminal,last_terminal_doc_timestamp,current_port,current_terminal,shipname,doc_timestamp,grt,teu,length,width
0,BREMERHAVEN,North Sea Terminal,2020-01-02 16:12:00.000,HAMBURG,Eurogate Container Terminal Hamburg,HEINRICH EHLER,2020-01-03 06:20:00.000,17488,1421,168.11,26.80
1,TILBURY,London Container Terminal,2020-01-01 21:56:00.000,HAMBURG,C. Steinweg Multipurpose Terminal,HENNEKE RAMBOW,2020-01-03 07:45:00.000,9981,868,134.40,22.74
2,ROTTERDAM MAASVLAKTE,Rotterdam World Gateway Terminal,2020-01-02 12:06:00.000,HAMBURG,HHLA Container Terminal Burchardkai,CMA CGM TANGER,2020-01-03 20:20:00.000,9966,1118,147.80,23.28
3,ROTTERDAM WAALHAVEN,RST Waalhaven,2020-01-01 16:05:00.000,HAMBURG,HHLA Container Terminal Burchardkai,NIEVES B,2020-01-03 20:28:00.000,10318,1036,151.72,23.40
4,GDYNIA,Gdynia Container Terminal,2020-01-01 14:45:00.000,HAMBURG,HHLA Container Terminal Burchardkai,JUDITH,2020-01-04 05:05:00.000,16023,1440,170.02,25.19


# Goals and Deliverable


The goal of this task is to implement & evaluate the accuracy of a solution that predicts the terminal
that a vessel will call.
The product team claims that the history of terminals visited by a vessel in the past is a critical factor
that should be incorporated into the model.
Some important steps that your solution would be expected to address and describe are the following:
* What features have you finally selected and engineered for your modeling approach? What led you to these choices? Why & how have you processed them?
* Which features seem to be the most important & how did you evaluate their importance?
* Do your findings agree with the product team’s insights discussed above? Before developing a ML model, how would you evaluate the importance/predictive power of one of the productidentified features as an independent variable?
* What type of prediction/training model have you chosen and why?
* How well does your predictive solution perform in terms of predicting the terminal a vessel will call?
* What different metrics/graphs can you use in order to understand when & why the algorithm fails/succeeds?
* What would be your baseline (i.e. a “naive” approach) to compare against?